# Builds the Seg Map and Catalog; Then, matches catalogs to get SEDs

In [1]:
# General imports
import os
import numpy as np
import matplotlib.pyplot as plt

# Segmentation
from astropy.io import fits
from photutils.segmentation import SourceFinder, make_2dgaussian_kernel, SourceCatalog
from photutils.background import Background2D, MedianBackground
from astropy.convolution import convolve

# Catalog Writing
from astropy.table import Table, Column
from collections import defaultdict

os.chdir("/Users/keith/astr/research_astr/summer-roman-project/4) FOV0_sims/fits")
root = "GRS_FOV0_roll0_dx0_dy0_SCA1_direct_final.fits" # Change filename with this variable
direct_file = root
seg_file = "seg_{0}".format(root)
cat_file = "cat_{0}.detect.cat".format(root)

In [2]:
# Process direct image and produce segmentation map

# Open image
direct_fits = fits.open(direct_file)
data, header = (direct_fits[1].data, direct_fits[1].header)
direct_fits.close()

# Subtract background
bkg_estimator = MedianBackground()
bkg = Background2D(data, (511,511), filter_size=(7,7), bkg_estimator=bkg_estimator)
data -= bkg.background

# Convolve image
kernel = make_2dgaussian_kernel(3.0, 5)
convolved_data = convolve(data, kernel)

# Instantiate the SourceFinder and set threshold
finder = SourceFinder(npixels=7, nlevels=32, contrast=0.01)
threshold = 1.5 * bkg.background_rms

seg_map = finder(convolved_data, threshold)

# Save seg_map as fits
fits.writeto(seg_file, np.rot90(seg_map, k=3), header=header,overwrite=True)

Deblending:   0%|          | 0/6976 [00:00<?, ?it/s]

In [3]:
# Create Catalog
cat = SourceCatalog(data, seg_map, convolved_data=convolved_data)

# Grizli expects these later
cat.add_extra_property('id', np.cast[int](cat.label), overwrite=True)
cat.add_extra_property('x_flt', cat.xcentroid, overwrite=True)
cat.add_extra_property('y_flt', cat.ycentroid, overwrite=True)

# Calculate flux using a reference object (reference object info hardcoded)
mag = -2.5 * np.log10(cat.segment_flux/24.614461302757263) + 23.02
cat.add_extra_property('mag_approx', mag, overwrite=True)

# Convert cumbersome catalog into usable table
columns = ['id', 'x_flt', 'y_flt', 'mag_approx', 'segment_flux']
tbl = cat.to_table(columns)

In [4]:
# Read in Table
hlss_tbl = Table.read("/Users/keith/astr/research_astr/FOV0/catalogs/MOT_SCA1_roll_0_dither_0x_0y_cut_zcut.txt",
                      format='ascii')
                    
# Instantiate Empty Column object; add column to tbl; repeat for all required columns
column = Column(name="mag", length=len(tbl), dtype=float)
tbl.add_column(column)

column = Column(name="hlss_id", length=len(tbl), dtype=int)
tbl.add_column(column)             

column = Column(name="distance", length=len(tbl), dtype=float)
tbl.add_column(column)       

column = Column(name="z", length=len(tbl), dtype=float)
tbl.add_column(column)   

column = Column(name="MODIMAGE", length=len(tbl), dtype=int)
tbl.add_column(column) 

# This has to be done a different way, or it'll only take one character
column = Column(data=np.empty(len(tbl)), name="SED", dtype=str)
tbl.add_column(column)

# Adding this for patches.ipynb
column = Column(name="A_IMAGE", length=len(tbl), dtype=float)
tbl.add_column(column) 

column = Column(name="B_IMAGE", length=len(tbl), dtype=float)
tbl.add_column(column) 

column = Column(name="THETA_IMAGE", length=len(tbl), dtype=float)
tbl.add_column(column) 

# Delete unused variable
del column

In [5]:
# Find potential matches by comparing seg_map identified objects to MOT expected objects
for ii, object in enumerate(tbl):
    
    # Set coord variables
    tbl_x, tbl_y = object["x_flt"], object["y_flt"]
    hlss_x, hlss_y = hlss_tbl["X_IMAGE"], hlss_tbl["Y_IMAGE"]

    # Calculate distance squared (square rooting is slow)
    d2d_squared = (tbl_x - hlss_x)**2 + (tbl_y - hlss_y)**2

    min_d2d = np.min(d2d_squared) # Find smallest distance
    min_match_dist = 10**2 # Set smallest acceptable distance for correctly identified objects

    # if object is correct
    if min_d2d < min_match_dist:

        # Find index of closes match
        min_index = np.argmin(d2d_squared)

        # Store match
        tbl["hlss_id"][ii] = hlss_tbl["NUMBER"][min_index]
        tbl["distance"][ii] = min_d2d

        # Store match's info from the HLSS catalog
        tbl["SED"][ii] = "SED:rest:gal.{0}.fits".format(hlss_tbl['SPECTEMP'][min_index])
        tbl["MODIMAGE"][ii] = hlss_tbl["MODIMAGE"][min_index]
        tbl['z'][ii] = hlss_tbl['Z'][min_index]
        tbl["mag"][ii] = hlss_tbl["MAG_F1500W"][min_index]

        # Adding this for patches.ipynb
        tbl["A_IMAGE"][ii] = hlss_tbl["A_IMAGE"][min_index]
        tbl["B_IMAGE"][ii] = hlss_tbl["B_IMAGE"][min_index]
        tbl["THETA_IMAGE"][ii] = hlss_tbl["THETA_IMAGE"][min_index]


matches = len(np.where(tbl["hlss_id"] != 0)[0])
print("# of matches: ", matches)

# of matches:  8195


In [6]:
# Identify MOT expected objects assign to two or more seg_map identified objects
D = defaultdict(list)

for row_num, hlss_id in enumerate(tbl['hlss_id']):
    D[hlss_id].append(row_num)
D = {k:v for k,v in D.items() if len(v)>1}

print("# of double matched objects: ", len(D))

# Identify farthest duplicates; assume those should be removed
remove = []
for key in D.keys():
    row_nums = D[key]
    keep = row_nums[np.argmin([tbl[ii]["distance"] for ii in row_nums])]
    remove.append([ii for ii in row_nums if ii != keep])

# Unmatch Duplicates
for row_num in remove:
    tbl["hlss_id"][row_num] = 0.0
    tbl["distance"][row_num] = 0.0
    tbl["SED"][row_num] = 0.0

# Keep only matched objects
tbl = tbl[np.where(tbl["hlss_id"] != 0.0)]

# Save Catalog
if os.path.exists(cat_file):
            os.remove(cat_file)
tbl.write(cat_file, format="ascii.ecsv")

# of double matched objects:  18
